# Make predictions for all models

In [7]:
# define the directory where the data is
data_path = "/home/jcapela/ec_number_prediction_version_2/ec_numbers_prediction/required_data_ec_number_paper/"

It is worth noting that we renamed the name of the folders containing the features just to make it easier to load them. We took it directly from the pipelines generated before. 

In [ ]:
from plants_sm.data_structures.dataset import SingleInputDataset
from plants_sm.models.pytorch_model import PyTorchModel


models = [("DeepEC", "one_hot_encoding", f"{data_path}/models/DeepEC_merged_merged"), 
          ("DSPACE", "one_hot_encoding", f"{data_path}/models/DSPACE_merged_merged"),
          ("esm2_3b", "", f"{data_path}/models/DNN_esm2_t36_3B_UR50D_trial_2_merged"), 
          ("prot_bert", "", f"{data_path}/models/DNN_prot_bert_vectors_trial_2_merged"), 
          ("esm1b", "", f"{data_path}/models/DNN_esm1b_t33_650M_UR50S_trial_4_merged"), 
         ]


test_dataset = SingleInputDataset.from_csv(f"{data_path}/data/test.csv",
                                            instances_ids_field="accession", representation_field="sequence",
                                            labels_field=slice(8, 2779))


for model_name, encoding, path in models:

    model = PyTorchModel.load(path)
    if encoding == "one_hot_encoding":
        test_dataset.load_features(f"{data_path}/features/test_one_hot/")
    else:
        test_dataset.load_features(f"{data_path}/features/test_{model_name}/")
    predictions = model.predict(test_dataset)
    # save predictions
    import pickle
    
    with open(f"{data_path}/predictions/predictions_{model_name}.pkl", "wb") as f:
        pickle.dump(predictions, f)